In [47]:
import pandas as pd
n_movies = pd.read_csv('n_movies.csv')
n_movies.head()

,title,year,certificate,duration,genre,rating,description,stars,votes
0,Cobra Kai,(2018– ),TV-14,30 min,"Action, Comedy, Drama",8.5,Decades after their 1984 All Valley Karate Tou...,"['Ralph Macchio, ', 'William Zabka, ', 'Courtn...","177,031"
1,The Crown,(2016– ),TV-MA,58 min,"Biography, Drama, History",8.7,Follows the political rivalries and romance of...,"['Claire Foy, ', 'Olivia Colman, ', 'Imelda St...","199,885"
2,Better Call Saul,(2015–2022),TV-MA,46 min,"Crime, Drama",8.9,The trials and tribulations of criminal lawyer...,"['Bob Odenkirk, ', 'Rhea Seehorn, ', 'Jonathan...","501,384"
3,Devil in Ohio,(2022),TV-MA,356 min,"Drama, Horror, Mystery",5.9,When a psychiatrist shelters a mysterious cult...,"['Emily Deschanel, ', 'Sam Jaeger, ', 'Gerardo...","9,773"
4,Cyberpunk: Edgerunners,(2022– ),TV-MA,24 min,"Animation, Action, Adventure",8.6,A Street Kid trying to survive in a technology...,"['Zach Aguilar, ', 'Kenichiro Ohashi, ', 'Emi ...","15,413"


In [48]:
n_movies['genre'] = n_movies['genre'].str.split(', ')

In [49]:
wrangled_n_movies = n_movies.loc[n_movies["rating"].notnull()]
# wrangled_n_movies.head()

In [45]:
netflix_titles = pd.read_csv("netflix_titles.csv")
netflix_titles.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [52]:
merged_data = netflix_titles.merge(wrangled_n_movies, how='inner', left_on='title', right_on='title')
print(merged_data)

     show_id     type                  title                 director  \
0         s1    Movie   Dick Johnson Is Dead          Kirsten Johnson   
1         s2  TV Show          Blood & Water                      NaN   
2         s2  TV Show          Blood & Water                      NaN   
3         s3  TV Show              Ganglands          Julien Leclercq   
4         s4  TV Show  Jailbirds New Orleans                      NaN   
...      ...      ...                    ...                      ...   
4637   s8776  TV Show        Yeh Meri Family                      NaN   
4638   s8778    Movie              Yes or No  Saratswadee Wongsomphet   
4639   s8789    Movie           You Carry Me               Ivona Juka   
4640   s8803    Movie                 Zodiac            David Fincher   
4641   s8804  TV Show            Zombie Dumb                      NaN   

                                                   cast  \
0                                                   NaN   
1    